<a href="https://colab.research.google.com/github/crashidian/Emotionally-Intelligent-Chatbot/blob/main/emotionallyintelligentchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch torch_geometric transformers networkx scikit-learn spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [ ]:
pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 61.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import networkx as nx
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

In [ ]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# Load SpaCy for NLP tasks
try:
    nlp = spacy.load("en_core_web_sm")
except ImportError:
    print("Error: SpaCy model 'en_core_web_sm' not found.")
    print("Please run the following command in your terminal:")
    print("python -m spacy download en_core_web_sm")
    print("Then restart this script.")
    exit(1)

In [ ]:
# Load GPT-2 model
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt2_model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# GNN Model for context and storytelling
class GNNModel(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, num_node_features)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        return self.conv3(x, edge_index)

In [ ]:
# Initialize GNN model
gnn_model = GNNModel(num_node_features=50, hidden_channels=64).to(device)
gnn_optimizer = torch.optim.Adam(gnn_model.parameters(), lr=0.01)

In [ ]:
# Create and maintain knowledge graph
knowledge_graph = nx.Graph()

In [ ]:
# TF-IDF vectorizer for text representation
tfidf = TfidfVectorizer(max_features=50)

In [ ]:
# Function to update knowledge graph
def update_knowledge_graph(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    for entity in entities:
        if entity not in knowledge_graph:
            knowledge_graph.add_node(entity, embedding=torch.rand(50, device=device))
    for i, entity1 in enumerate(entities):
        for entity2 in entities[i+1:]:
            if not knowledge_graph.has_edge(entity1, entity2):
                knowledge_graph.add_edge(entity1, entity2)

In [ ]:
# Function to get graph embeddings
def get_graph_embeddings():
    if not knowledge_graph.nodes:
        return torch.tensor([], device=device), torch.tensor([], device=device)

    node_features = torch.stack([knowledge_graph.nodes[node]['embedding'] for node in knowledge_graph.nodes])
    edges = list(knowledge_graph.edges())
    if not edges:
        return node_features, torch.tensor([], device=device)
    edge_index = torch.tensor([[list(knowledge_graph.nodes()).index(edge[0]),
                                list(knowledge_graph.nodes()).index(edge[1])]
                               for edge in edges], device=device).t().contiguous()
    return node_features, edge_index

In [ ]:
# Emotion recognition with psychological context
def recognize_emotion(text):
    doc = nlp(text)

    # Basic emotion detection
    emotion_keywords = {
        'joy': ['happy', 'joy', 'excited', 'glad'],
        'sadness': ['sad', 'upset', 'depressed', 'unhappy'],
        'anger': ['angry', 'mad', 'furious', 'annoyed'],
        'fear': ['afraid', 'scared', 'terrified', 'anxious'],
        'surprise': ['surprised', 'shocked', 'amazed'],
    }

    detected_emotion = 'neutral'
    for emotion, keywords in emotion_keywords.items():
        if any(word in text.lower() for word in keywords):
            detected_emotion = emotion
            break

    # Psychological context
    sentiment = doc.sentiment
    important_entities = [ent.text for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'EVENT']]

    return {
        'basic_emotion': detected_emotion,
        'sentiment': sentiment,
        'important_entities': important_entities
    }

In [ ]:
# Generate empathetic response using GPT-2
def generate_empathetic_response(user_input, emotion_info, conversation_history):
    try:
        # Update knowledge graph
        update_knowledge_graph(user_input)

        # Get graph embeddings
        node_features, edge_index = get_graph_embeddings()

        print("Node features shape:", node_features.shape)
        print("Edge index shape:", edge_index.shape)

        # Use GNN to process graph if we have valid data
        if node_features.nelement() > 0 and edge_index.nelement() > 0:
            with torch.no_grad():
                gnn_output = gnn_model(node_features, edge_index)
        else:
            print("Warning: Not enough data for GNN processing")
            gnn_output = torch.tensor([], device=device)

        # Create context-aware prompt
        context = "Based on our conversation, I understand that:\n"
        context += f"- You're feeling {emotion_info.get('basic_emotion', 'neutral')}\n"
        context += f"- The sentiment of your message is {emotion_info.get('sentiment', 'neutral')}\n"
        important_entities = emotion_info.get('important_entities', [])
        if important_entities:
            context += f"- Important aspects mentioned: {', '.join(important_entities)}\n"
        context += f"Considering this context, respond empathetically to: {user_input}"

        full_prompt = f"{conversation_history}\nHuman: {context}\nAI:"

        # Generate response using GPT-2
        input_ids = tokenizer.encode(full_prompt, return_tensors='pt', truncation=True, max_length=1024).to(device)
        attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)

        output = gpt2_model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=min(len(input_ids[0]) + 100, 1024),
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
        )

        response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Extract AI's response
        ai_response = response.split("AI:")[-1].strip()

        return ai_response

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return "I apologize, but I encountered an error while processing your input. Could you please try rephrasing your message?"


In [ ]:
# Train GNN model (simplified)
def train_gnn():
    gnn_model.train()
    node_features, edge_index = get_graph_embeddings()
    if node_features.nelement() > 0 and edge_index.nelement() > 0:
        gnn_optimizer.zero_grad()
        out = gnn_model(node_features, edge_index)
        loss = F.mse_loss(out, node_features)
        loss.backward()
        gnn_optimizer.step()

In [ ]:
# Interactive chat function
def interactive_chat():
    print("Welcome to the GPT-2 GNN Empathetic Chatbot!")
    print("Type 'quit', 'exit', or 'bye' to end the conversation.")

    conversation_history = ""

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Chatbot: I understand you want to end our conversation. I hope our chat was helpful. Take care, and remember that it's okay to reach out if you need support. Goodbye!")
            break

        emotion_info = recognize_emotion(user_input)
        response = generate_empathetic_response(user_input, emotion_info, conversation_history)
        print(f"Chatbot: {response}")

        # Update conversation history
        conversation_history += f"\nHuman: {user_input}\nAI: {response}"

        # Keep conversation history to a reasonable size
        conversation_history = "\n".join(conversation_history.split("\n")[-20:])

        # Train GNN on updated knowledge graph
        train_gnn()

In [ ]:
# Main execution
if __name__ == "__main__":
    interactive_chat()

Welcome to the GPT-2 GNN Empathetic Chatbot!
Type 'quit', 'exit', or 'bye' to end the conversation.
You: Hello. I am feeling very happy today because I woke up early and completed all of my chores.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Node features shape: torch.Size([1, 50])
Edge index shape: torch.Size([0])
Chatbot: This sentiment is not 0.
I'd like to know what is the sentiment that makes you feel happy, if any?
Thank you.
You: The reason why I am feeling happy is that I finished all of my housework before the start of my work day, including the sweeping, vacuuming, litter box, and other stuff.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Node features shape: torch.Size([1, 50])
Edge index shape: torch.Size([0])
Chatbot: Now, it's time to give you a task. What are you feeling right now? The result of the task you want to complete? Yes or no. You may use the "yes" or "no" response. A "YES" means that you're satisfied with the result. If you choose "NO," you'll have to work harder to finish your task, but you can choose to start working on it again later. When you decide to take a break, give yourself a chance to
You: I am feeling happy. The result of the task makes me happy.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Node features shape: torch.Size([1, 50])
Edge index shape: torch.Size([0])
Chatbot: Well, you chose "I am happy."
So, now we know that happiness is a state of mind, which is something we can influence by doing things. We can control our emotions, by feeling them, or by simply letting them go. In other words, we don't have any control over our feelings. But we do have control of our actions. Do you see, that when you get angry, the emotion is going to make you angry. So we must avoid anger. However,
You: quit
Chatbot: I understand you want to end our conversation. I hope our chat was helpful. Take care, and remember that it's okay to reach out if you need support. Goodbye!
